In [1]:
import pandas as pd
import pandas.io.sql as psql
import os
import sys
from datetime import date
current_date = date.today()
#%load_ext autotime
import re
import csv
import pandas as pd
import numpy as np
import os
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
#from matplotlib_venn import venn2, venn3
import warnings
import time
import datetime as dt
import itertools
from more_itertools import unique_everseen
warnings.filterwarnings("ignore")
#import pandasql as ps
import seaborn as sns
import json
from ast import literal_eval
import ast
from fuzzywuzzy import process
from nltk.corpus import words

## Getting Encounter quantities per patient and in general

In [63]:
e = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\padded_matches_7_14.csv")
n = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\not_padded_matches_7_14.csv")

In [64]:
e = e.drop(columns = ['Unnamed: 0.1.1.1.1'], axis = 1) #only run if runtime reset

In [65]:
e

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,...,regex_match,match_count,pruned_regex_location,merged_row_location,merged_regex_sent,merged_regex_match,padded_merged_row_location,padded_merged_regex_sent,padded_char_count,note_char_count
0,0,0,0,0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,Office Visit,...,['cerebral'],1,"[(100, 109)]","[(5750, 5958)]",------- ncer. She passed secondary to this. He...,[' cerebral'],"(5457, 6251)",ked at HMS in the department of psychobiology ...,794,19217
1,1,1,1,1,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,Office Visit,...,['memory'],1,"[(100, 107)]","[(3543, 3749)]",------- a. Skin: No rashes or lesions. Neurolo...,[' memory'],"(3248, 4044)",or lymphadenopathy. Chest: Clear to auscultati...,796,5850
2,2,2,2,14,100000272,10040002098,Z6352398,3.242118e+09,2019-03-14 00:00:00.0000000,Office Visit,...,['dementia'],1,"[(100, 109)]","[(8641, 8849)]","------- ractures, obesity, insulin resistance,...",[' dementia'],"(8345, 9145)",ession alone in the meta-analysis. We discusse...,800,13298
3,3,3,3,15,100000272,10040002098,Z6352398,3.322836e+09,2020-12-04 00:00:00.0000000,Telemedicine - audio only,...,['MOCA'],1,"[(100, 105)]","[(948, 1152), (4778, 4987), (5004, 5331)]",------- accept for this week when he ran out ...,"[' moca', ' cognition', ' memory']","[(930, 1152), (4778, 4987), (5004, 5349)]",------- ing tylenol really accept for this we...,803,8711
4,4,4,4,20,100000272,10040002098,Z6352398,3.325561e+09,2020-09-18 00:00:00.0000000,Telemedicine - audio only,...,['cognition'],1,"[(100, 110)]","[(5367, 5576), (5593, 5812)]",------- ue Tylenol 1g q8 as needed - If pain w...,"[' cognition', ' memory']","[(5188, 5576), (5593, 5991)]",------- in is related to cancer. Reassuringly...,804,9145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238914,48354,60866,60866,278140,113722742,10127721883,Z16337903,3.215952e+09,2018-09-05 00:00:00.0000000,Office Visit,...,"[""cerebellar""]",1,"[(91, 102)]","[(10379, 10589), (10838, 11046), (11051, 11294)]",------- INDINGS: The brain parenchyma demonstr...,"[' cerebral', ' cerebellar']","[(10320, 10589), (10838, 11046), (11051, 11353)]",------- pulse sequence(s). MRA of the neck w...,806,17293
238915,48355,60867,60867,278149,113722742,10127721883,Z16337903,3.217899e+09,2018-10-03 00:00:00.0000000,Office Visit,...,"[""cognition""]",1,"[(96, 106)]","[(3719, 3928), (10689, 10895)]",------- ROS performed and are negative. The pa...,"[' cognition', ' memory']","[(3531, 3928), (10689, 11083)]",------- ONITORING LT4: TSH Date Value Ref Ran...,809,12288
238916,48356,60868,60868,278154,113722742,10127721883,Z16337903,3.223494e+09,2018-10-31 00:00:00.0000000,Office Visit,...,"[""Cognition""]",1,"[(99, 109)]","[(1982, 2191)]","------- (R) gait status, as well as getting u...",[' cognition'],"[(1690, 2483)]",e patient felt much more comfortable with and ...,793,4006
238917,48357,60869,60869,278163,113722742,10127721883,Z16337903,3.329813e+09,2020-11-04 00:00:00.0000000,Office Visit,...,"[""cognition""]",1,"[(94, 104)]","[(3580, 3789)]",------- her lesions. Normal turgor. [Psych]: A...,[' cognition'],"[(3071, 3871)]","y Social History Narrative Divorced, 1 daughte...",800,3872


In [67]:
n["sequence_level_regex_match"] = n["merged_regex_sent"].to_list()

In [69]:
n.columns = e.columns

In [70]:
df = pd.concat([e, n])

In [71]:
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,...,regex_match,match_count,pruned_regex_location,merged_row_location,merged_regex_sent,merged_regex_match,padded_merged_row_location,padded_merged_regex_sent,padded_char_count,note_char_count
0,0,0,0,0.0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,Office Visit,...,['cerebral'],1,"[(100, 109)]","[(5750, 5958)]",------- ncer. She passed secondary to this. He...,[' cerebral'],"(5457, 6251)",ked at HMS in the department of psychobiology ...,794,19217.0
1,1,1,1,1.0,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,Office Visit,...,['memory'],1,"[(100, 107)]","[(3543, 3749)]",------- a. Skin: No rashes or lesions. Neurolo...,[' memory'],"(3248, 4044)",or lymphadenopathy. Chest: Clear to auscultati...,796,5850.0
2,2,2,2,14.0,100000272,10040002098,Z6352398,3.242118e+09,2019-03-14 00:00:00.0000000,Office Visit,...,['dementia'],1,"[(100, 109)]","[(8641, 8849)]","------- ractures, obesity, insulin resistance,...",[' dementia'],"(8345, 9145)",ession alone in the meta-analysis. We discusse...,800,13298.0
3,3,3,3,15.0,100000272,10040002098,Z6352398,3.322836e+09,2020-12-04 00:00:00.0000000,Telemedicine - audio only,...,['MOCA'],1,"[(100, 105)]","[(948, 1152), (4778, 4987), (5004, 5331)]",------- accept for this week when he ran out ...,"[' moca', ' cognition', ' memory']","[(930, 1152), (4778, 4987), (5004, 5349)]",------- ing tylenol really accept for this we...,803,8711.0
4,4,4,4,20.0,100000272,10040002098,Z6352398,3.325561e+09,2020-09-18 00:00:00.0000000,Telemedicine - audio only,...,['cognition'],1,"[(100, 110)]","[(5367, 5576), (5593, 5812)]",------- ue Tylenol 1g q8 as needed - If pain w...,"[' cognition', ' memory']","[(5188, 5576), (5593, 5991)]",------- in is related to cancer. Reassuringly...,804,9145.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40300,60817,60817,277874,NaN,113705759,10127549029,Z16320629,3.285425e+09,2020-04-28 00:00:00.0000000,Telemedicine - audio only,...,"[""memory""]",1,"[(97, 104)]","[(1346, 1552), (1603, 1809), (5360, 5569), (59...","------- se ""she took a break"" and wants to dis...","[' memory', ' cognition', ' cerebral', ' cereb...","[(538, 547), (1626, 1635), (1844, 1853), (108,...","------- se ""she took a break"" and wants to dis...",1949,11314.0
40301,60818,60818,277883,NaN,113705759,10127549029,Z16320629,3.305416e+09,2020-09-22 00:00:00.0000000,Telemedicine - audio only,...,"[""memory""]",1,"[(99, 106)]","[(1165, 1371), (4255, 4464), (4876, 5084), (56...","------- all of her medications, including asp...","[' memory', ' cognition', ' cerebral', ' cereb...","[(323, 332), (1411, 1420), (1629, 1638), (108,...","------- all of her medications, including asp...",1734,10113.0
40302,60819,60819,277896,NaN,113705759,10127549029,Z16320629,3.327430e+09,2021-01-25 00:00:00.0000000,Telemedicine,...,"[""cognition""]",1,"[(98, 108)]","[(194, 403), (444, 653), (1326, 1532), (4453, ...","------- ? Patient, Chart, daughter Olga Mode o...","[' cognition', ' cerebral', ' memory', ' cereb...","[(2816, 2824), (108, 117), (326, 335), (974, 9...","------- ? Patient, Chart, daughter Olga Mode o...",3009,12320.0
40303,60820,60820,277912,NaN,113705759,10127549029,Z16320629,3.349755e+09,2021-04-20 00:00:00.0000000,Telemedicine,...,"[""cognition""]",1,"[(98, 108)]","[(202, 411), (452, 661), (3626, 3875), (4946, ...","------- ? Patient, Chart, daughter Olga Mode o...","[' cognition', ' cerebral', ' cognitive impair...","[(2103, 2111), (108, 117), (326, 335), (544, 5...","------- ? Patient, Chart, daughter Olga Mode o...",2835,11250.0


In [72]:
speciality = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\specialty_visit_encounters.csv")
pcp = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Mayuresh\APOE33\pcp_visit_encounters.csv")

In [73]:
df = pd.merge(df, pcp, left_on = 'PatientEncounterID', right_on = 'PatientEncounterID', how = 'left')

In [74]:
df = pd.merge(df, speciality, left_on = 'PatientEncounterID', right_on = 'PatientEncounterID', how = 'left')

In [75]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,...,merged_regex_sent,merged_regex_match,padded_merged_row_location,padded_merged_regex_sent,padded_char_count,note_char_count,PCPID,EncounterEpicProviderID,pcp_flag,SpecialtyDSC
0,0,0,0,0.0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,Office Visit,...,------- ncer. She passed secondary to this. He...,[' cerebral'],"(5457, 6251)",ked at HMS in the department of psychobiology ...,794,19217.0,NaN,1521494,0,NaN
1,1,1,1,1.0,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,Office Visit,...,------- a. Skin: No rashes or lesions. Neurolo...,[' memory'],"(3248, 4044)",or lymphadenopathy. Chest: Clear to auscultati...,796,5850.0,NaN,1012865,0,NaN
2,2,2,2,14.0,100000272,10040002098,Z6352398,3.242118e+09,2019-03-14 00:00:00.0000000,Office Visit,...,"------- ractures, obesity, insulin resistance,...",[' dementia'],"(8345, 9145)",ession alone in the meta-analysis. We discusse...,800,13298.0,1002888.0,1668416,0,NaN
3,3,3,3,15.0,100000272,10040002098,Z6352398,3.322836e+09,2020-12-04 00:00:00.0000000,Telemedicine - audio only,...,------- accept for this week when he ran out ...,"[' moca', ' cognition', ' memory']","[(930, 1152), (4778, 4987), (5004, 5349)]",------- ing tylenol really accept for this we...,803,8711.0,1002888.0,1746848,0,NaN
4,4,4,4,20.0,100000272,10040002098,Z6352398,3.325561e+09,2020-09-18 00:00:00.0000000,Telemedicine - audio only,...,------- ue Tylenol 1g q8 as needed - If pain w...,"[' cognition', ' memory']","[(5188, 5576), (5593, 5991)]",------- in is related to cancer. Reassuringly...,804,9145.0,1002888.0,1746848,0,NaN


In [89]:
len(df["PatientID"].unique())

16428

In [103]:
df_clone = df.drop_duplicates(subset='PatientEncounterID', keep="first")

In [105]:
len(df_clone)

178994

In [104]:
encounter_counts = df_clone.groupby(["EMPI"])['PatientEncounterID'].count().reset_index()
encounter_counts

,EMPI,PatientEncounterID
0,100000272,9
1,100000521,6
2,100000623,3
3,100000894,5
4,100000920,3
...,...,...
16423,113714475,2
16424,113716850,4
16425,113722466,5
16426,113722742,11


In [106]:
encounter_counts["PatientEncounterID"].describe()

count    16428.000000
mean        10.895666
std         14.880333
min          1.000000
25%          2.000000
50%          6.000000
75%         13.000000
max        316.000000
Name: PatientEncounterID, dtype: float64

In [150]:
encounter_counts.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\encounter_counts.csv", index= False)

In [76]:
speciality_counts = df.groupby(["EMPI"])['SpecialtyDSC'].count().reset_index()

In [77]:
speciality_counts.head()

,EMPI,SpecialtyDSC
0,100000272,0
1,100000521,0
2,100000623,0
3,100000894,0
4,100000920,0


In [83]:
speciality_counts["SpecialtyDSC"].describe()

count    16428.000000
mean         1.673545
std          4.634825
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         87.000000
Name: SpecialtyDSC, dtype: float64

In [88]:
speciality_counts["SpecialtyDSC"].median()

0.0

In [92]:
speciality_counts["SpecialtyDSC"].sum()

27493

In [78]:
pcp_counts = df[df["pcp_flag"] == 1].groupby(["EMPI"])["pcp_flag"].count().reset_index()

In [81]:
pcp_counts.head()

,EMPI,pcp_flag
0,100000521,1
1,100001150,3
2,100001521,14
3,100001768,8
4,100001873,3


In [84]:
pcp_counts["pcp_flag"].describe()

count    7587.000000
mean        5.251219
std         5.630849
min         1.000000
25%         2.000000
50%         4.000000
75%         6.000000
max        94.000000
Name: pcp_flag, dtype: float64

In [94]:
pcp_counts["pcp_flag"].median()

4.0

In [91]:
pcp_counts["pcp_flag"].sum()

39841

In [151]:
speciality_counts.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\speciality_counts.csv", index = False)

In [152]:
pcp_counts.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\pcp_counts.csv", index = False)

## Recalculating Keyword Matches

In [111]:
unique_empi = df["PatientID"].unique().tolist()

In [112]:
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,...,merged_regex_sent,merged_regex_match,padded_merged_row_location,padded_merged_regex_sent,padded_char_count,note_char_count,PCPID,EncounterEpicProviderID,pcp_flag,SpecialtyDSC
0,0,0,0,0.0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,Office Visit,...,------- ncer. She passed secondary to this. He...,[' cerebral'],"(5457, 6251)",ked at HMS in the department of psychobiology ...,794,19217.0,NaN,1521494,0,NaN
1,1,1,1,1.0,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,Office Visit,...,------- a. Skin: No rashes or lesions. Neurolo...,[' memory'],"(3248, 4044)",or lymphadenopathy. Chest: Clear to auscultati...,796,5850.0,NaN,1012865,0,NaN
2,2,2,2,14.0,100000272,10040002098,Z6352398,3.242118e+09,2019-03-14 00:00:00.0000000,Office Visit,...,"------- ractures, obesity, insulin resistance,...",[' dementia'],"(8345, 9145)",ession alone in the meta-analysis. We discusse...,800,13298.0,1002888.0,1668416,0,NaN
3,3,3,3,15.0,100000272,10040002098,Z6352398,3.322836e+09,2020-12-04 00:00:00.0000000,Telemedicine - audio only,...,------- accept for this week when he ran out ...,"[' moca', ' cognition', ' memory']","[(930, 1152), (4778, 4987), (5004, 5349)]",------- ing tylenol really accept for this we...,803,8711.0,1002888.0,1746848,0,NaN
4,4,4,4,20.0,100000272,10040002098,Z6352398,3.325561e+09,2020-09-18 00:00:00.0000000,Telemedicine - audio only,...,------- ue Tylenol 1g q8 as needed - If pain w...,"[' cognition', ' memory']","[(5188, 5576), (5593, 5991)]",------- in is related to cancer. Reassuringly...,804,9145.0,1002888.0,1746848,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279219,60817,60817,277874,NaN,113705759,10127549029,Z16320629,3.285425e+09,2020-04-28 00:00:00.0000000,Telemedicine - audio only,...,"------- se ""she took a break"" and wants to dis...","[' memory', ' cognition', ' cerebral', ' cereb...","[(538, 547), (1626, 1635), (1844, 1853), (108,...","------- se ""she took a break"" and wants to dis...",1949,11314.0,1007065.0,1558180,0,NaN
279220,60818,60818,277883,NaN,113705759,10127549029,Z16320629,3.305416e+09,2020-09-22 00:00:00.0000000,Telemedicine - audio only,...,"------- all of her medications, including asp...","[' memory', ' cognition', ' cerebral', ' cereb...","[(323, 332), (1411, 1420), (1629, 1638), (108,...","------- all of her medications, including asp...",1734,10113.0,1007065.0,1558180,0,NaN
279221,60819,60819,277896,NaN,113705759,10127549029,Z16320629,3.327430e+09,2021-01-25 00:00:00.0000000,Telemedicine,...,"------- ? Patient, Chart, daughter Olga Mode o...","[' cognition', ' cerebral', ' memory', ' cereb...","[(2816, 2824), (108, 117), (326, 335), (974, 9...","------- ? Patient, Chart, daughter Olga Mode o...",3009,12320.0,1007065.0,1558180,0,NaN
279222,60820,60820,277912,NaN,113705759,10127549029,Z16320629,3.349755e+09,2021-04-20 00:00:00.0000000,Telemedicine,...,"------- ? Patient, Chart, daughter Olga Mode o...","[' cognition', ' cerebral', ' cognitive impair...","[(2103, 2111), (108, 117), (326, 335), (544, 5...","------- ? Patient, Chart, daughter Olga Mode o...",2835,11250.0,1007065.0,1558180,0,NaN


In [115]:
regex = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\APOE-SLAT\EDA\Getting_Data\keywords.csv")

k = regex["REGEX"].to_list()
c = regex["CASE"].to_list()
p_list = []

for i in range(len(k)):
    if (c[i] == 0):
        p_list.append(re.compile(k[i][5:], re.IGNORECASE))
    elif (c[i] == 1):
        p_list.append(re.compile(k[i]))
print(p_list)

[re.compile('\\bdementia\\b', re.IGNORECASE), re.compile('\\bcognition\\b', re.IGNORECASE), re.compile('\\bmemory\\b', re.IGNORECASE), re.compile('\\bMMSE\\b', re.IGNORECASE), re.compile('\\bMOCA\\b', re.IGNORECASE), re.compile('\\bAlzheimer\\b', re.IGNORECASE), re.compile('\\bcognitive\\s*impairment\\b', re.IGNORECASE), re.compile('(?-i:)\\bMCI\\b'), re.compile('\\bcerebellar\\b', re.IGNORECASE), re.compile('\\bNeurocognitive\\b', re.IGNORECASE), re.compile('\\blewy\\b', re.IGNORECASE), re.compile("\\bpick's\\b", re.IGNORECASE), re.compile('\\bCorticobasal\\b', re.IGNORECASE), re.compile('\\bcerebral\\b', re.IGNORECASE), re.compile('\\bcerebrovascular\\b', re.IGNORECASE), re.compile('\\bamnesia\\b', re.IGNORECASE), re.compile('(?-i:)\\bAD\\b'), re.compile('(?-i:)\\bLBD\\b')]


In [116]:
l = []
for note in tqdm(df["padded_merged_regex_sent"]):
    curr = []
    for p in (p_list):
        m = list(set(re.findall(p, note)))
        m = list(set(map(str.lower, m)))
        if (m != []):
            curr.append("".join(m))
    #print(curr)
    #print(l)
    l.append(str(curr))

df["padded_regex_match"] = l

100%|████████████████████████████████████████████████████████████████████████| 279224/279224 [03:03<00:00, 1520.78it/s]


In [117]:
cols =  ['dementia', 'cognition', 'memory', 'mmse', 'moca', 'alzheimer', 'cognitive impairment', 'mci', 'cerebellar', 'neurocognitive', 'lewy', "pick's",  'corticobasal', 'cerebral', 'cerebrovascular', 'amnesia', 'ad', 'lbd']
len(cols)

18

In [118]:
summary_stats  = pd.DataFrame(pd.np.empty((len(unique_empi), len(cols))) * pd.np.nan) 
summary_stats.columns = cols

In [119]:
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,EMPI,MRN,PatientID,PatientEncounterID,ContactDTS,EncounterTypeDSC,...,merged_regex_match,padded_merged_row_location,padded_merged_regex_sent,padded_char_count,note_char_count,PCPID,EncounterEpicProviderID,pcp_flag,SpecialtyDSC,padded_regex_match
0,0,0,0,0.0,100000272,10040002098,Z6352398,3.076070e+09,2015-06-17 00:00:00.0000000,Office Visit,...,[' cerebral'],"(5457, 6251)",ked at HMS in the department of psychobiology ...,794,19217.0,NaN,1521494,0,NaN,['cerebral']
1,1,1,1,1.0,100000272,10040002098,Z6352398,3.076214e+09,2015-07-16 00:00:00.0000000,Office Visit,...,[' memory'],"(3248, 4044)",or lymphadenopathy. Chest: Clear to auscultati...,796,5850.0,NaN,1012865,0,NaN,['memory']
2,2,2,2,14.0,100000272,10040002098,Z6352398,3.242118e+09,2019-03-14 00:00:00.0000000,Office Visit,...,[' dementia'],"(8345, 9145)",ession alone in the meta-analysis. We discusse...,800,13298.0,1002888.0,1668416,0,NaN,['dementia']
3,3,3,3,15.0,100000272,10040002098,Z6352398,3.322836e+09,2020-12-04 00:00:00.0000000,Telemedicine - audio only,...,"[' moca', ' cognition', ' memory']","[(930, 1152), (4778, 4987), (5004, 5349)]",------- ing tylenol really accept for this we...,803,8711.0,1002888.0,1746848,0,NaN,"['cognition', 'memory', 'moca']"
4,4,4,4,20.0,100000272,10040002098,Z6352398,3.325561e+09,2020-09-18 00:00:00.0000000,Telemedicine - audio only,...,"[' cognition', ' memory']","[(5188, 5576), (5593, 5991)]",------- in is related to cancer. Reassuringly...,804,9145.0,1002888.0,1746848,0,NaN,"['cognition', 'memory']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279219,60817,60817,277874,NaN,113705759,10127549029,Z16320629,3.285425e+09,2020-04-28 00:00:00.0000000,Telemedicine - audio only,...,"[' memory', ' cognition', ' cerebral', ' cereb...","[(538, 547), (1626, 1635), (1844, 1853), (108,...","------- se ""she took a break"" and wants to dis...",1949,11314.0,1007065.0,1558180,0,NaN,"['cognition', 'memory', 'cerebellar', 'cerebral']"
279220,60818,60818,277883,NaN,113705759,10127549029,Z16320629,3.305416e+09,2020-09-22 00:00:00.0000000,Telemedicine - audio only,...,"[' memory', ' cognition', ' cerebral', ' cereb...","[(323, 332), (1411, 1420), (1629, 1638), (108,...","------- all of her medications, including asp...",1734,10113.0,1007065.0,1558180,0,NaN,"['cognition', 'memory', 'cerebellar', 'cerebral']"
279221,60819,60819,277896,NaN,113705759,10127549029,Z16320629,3.327430e+09,2021-01-25 00:00:00.0000000,Telemedicine,...,"[' cognition', ' cerebral', ' memory', ' cereb...","[(2816, 2824), (108, 117), (326, 335), (974, 9...","------- ? Patient, Chart, daughter Olga Mode o...",3009,12320.0,1007065.0,1558180,0,NaN,"['dementia', 'cognition', 'memory', 'mci', 'ce..."
279222,60820,60820,277912,NaN,113705759,10127549029,Z16320629,3.349755e+09,2021-04-20 00:00:00.0000000,Telemedicine,...,"[' cognition', ' cerebral', ' cognitive impair...","[(2103, 2111), (108, 117), (326, 335), (544, 5...","------- ? Patient, Chart, daughter Olga Mode o...",2835,11250.0,1007065.0,1558180,0,NaN,"['dementia', 'cognition', 'moca', 'cognitive i..."


In [120]:
for i in tqdm(range(len(unique_empi))):
    keyword_list = list((df[df['PatientID'] == unique_empi[i]]['padded_regex_match'].values))
    #print("KL: ", keyword_list)
    match_tuples = []
    for j in range(len(keyword_list)):
        sub = "\', \'"
        x = list(eval(keyword_list[j]))# [2:len(keyword_list[j])-2]
        #print("X: ", x)
#         if sub in x:
#             #print("YES")
#             x = x.split("\', \'")
#             #print("X2: ", x)
#         else:
#             x = x.split("\", \"")
        for z in range(len(x)):
            match_tuples.append((x[z], 0))
    #print("MT: ", match_tuples)

    freq = {"dementia":0
        ,"cognition":0
        ,"cognition":0
        ,"memory":0
        ,"mmse":0
        ,"moca":0
        ,"alzheimer":0
        ,'cognitive impairment':0
        ,"mci":0
        ,"cerebellar":0
        ,"neurocognitive":0
        ,"lewy":0
        ,"pick's":0
        ,"corticobasal":0
        ,"cerebral":0
        ,"cerebrovascular":0
        ,"amnesia":0
        ,"ad": 0
        ,"lbd": 0
    }

    for k, v in match_tuples:   
        if (k != ''):
            if (k == 'cognitive impairmentcognitiveimpairment'):
                k = 'cognitive impairment'
                freq.update({str(k.lower()):int(freq[k.lower()] + 1)})
            else:
                freq.update({str(k.lower()):int(freq[k.lower()] + 1)})
            
    #print(freq.items())
    #print(summary_stats["dementia"])
    for k, v in freq.items():
        #print(str(k.lower()))
        summary_stats[str(k.lower())][i] = int(v)

100%|████████████████████████████████████████████████████████████████████████████| 16428/16428 [09:40<00:00, 28.30it/s]


In [121]:
summary_stats

,dementia,cognition,memory,mmse,moca,alzheimer,cognitive impairment,mci,cerebellar,neurocognitive,lewy,pick's,corticobasal,cerebral,cerebrovascular,amnesia,ad,lbd
0,2.0,4.0,7.0,0.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0
3,1.0,3.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16423,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
16424,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
16425,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
16426,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [122]:
summary_stats.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\summary_stats_7_14.csv")

In [123]:
summary_stats = pd.read_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\summary_stats_7_14.csv")

In [125]:
summary_stats["Sum"].describe()

count    16429.000000
mean        26.218408
std         53.763780
min          1.000000
25%          3.000000
50%          9.000000
75%         25.000000
max       1423.000000
Name: Sum, dtype: float64

In [126]:
summary_stats["Sum"].median()

9.0

In [140]:
match_counts = summary_stats.loc[[16428]].T
match_counts.columns = ['Sum']

In [142]:
match_counts.sort_values(by = ['Sum'], ascending=False)

,Sum
memory,109218.000000
cognition,87655.000000
dementia,51034.000000
cerebral,45886.000000
cerebrovascular,36370.000000
cerebellar,26863.000000
alzheimer,20581.000000
cognitive impairment,20267.000000
moca,9767.000000
neurocognitive,7711.000000


In [147]:
match_counts = match_counts.drop(labels = ['Sum', 'Unnamed: 0'], axis = 0)

In [148]:
match_counts

,Sum
dementia,51034.0
cognition,87655.0
memory,109218.0
mmse,2134.0
moca,9767.0
alzheimer,20581.0
cognitive impairment,20267.0
mci,3889.0
cerebellar,26863.0
neurocognitive,7711.0


In [149]:
match_counts.to_csv(r"C:\Users\MIND_DS\Dropbox (Partners HealthCare)\NLP\Tanish\BigDataSets\Regex_match\FINAL\match_counts.csv")